In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.features import shapes
from rasterio.transform import from_origin
import geopandas as gpd
from scipy.optimize import curve_fit
from scipy.signal import argrelextrema
import glob


In [2]:
# Read the Parquet file
df = pd.read_parquet("wheat_pixel.parquet")


In [3]:
df

,latitude,longitude,timestep_1,timestep_2,timestep_3,timestep_4,timestep_5,timestep_6,timestep_7,timestep_8,timestep_9,timestep_10,timestep_11,timestep_12,timestep_13,timestep_14
0,30.240077,77.714064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30.240077,77.714154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30.240077,77.714243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30.240077,77.714333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30.240077,77.714423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50315695,29.580804,78.329410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50315696,29.580804,78.329500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50315697,29.580804,78.329589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50315698,29.580804,78.329679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df = df.replace(0, pd.NA)
data = df.dropna()
data

,latitude,longitude,timestep_1,timestep_2,timestep_3,timestep_4,timestep_5,timestep_6,timestep_7,timestep_8,timestep_9,timestep_10,timestep_11,timestep_12,timestep_13,timestep_14
2689,30.240077,77.955621,0.524752,0.553589,0.522286,0.59204,0.300691,0.313164,0.567788,0.483366,0.540415,0.49782,0.447154,0.479344,0.312207,0.38066
2690,30.240077,77.955711,0.523729,0.540253,0.470952,0.520866,0.2575,0.313164,0.525828,0.478057,0.517939,0.461378,0.426923,0.532698,0.319149,0.424261
2692,30.240077,77.955890,0.467526,0.496446,0.454775,0.590361,0.243655,0.3,0.638424,0.62001,0.657407,0.640175,0.560887,0.571265,0.380217,0.629849
2693,30.240077,77.955980,0.36811,0.460727,0.45828,0.569362,0.138039,0.217753,0.563296,0.518771,0.569691,0.54326,0.406413,0.456286,0.336175,0.436864
9547,30.239987,77.955890,0.574109,0.549407,0.537442,0.651181,0.438735,0.406828,0.725471,0.628253,0.65868,0.610591,0.486381,0.545975,0.417304,0.619677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,29.580983,77.971431,0.465719,0.540412,0.66215,0.68348,0.573653,0.366557,0.473824,0.467172,0.241561,0.195604,0.236601,0.145597,0.084599,0.15009
50298002,29.580983,77.971611,0.474039,0.578114,0.665991,0.697608,0.577744,0.381807,0.450522,0.446207,0.287449,0.236979,0.270392,0.131527,0.082659,0.173953
50298014,29.580983,77.972689,0.243697,0.242347,0.280808,0.365385,0.501665,0.422971,0.66826,0.757663,0.735261,0.712784,0.654444,0.21504,0.280859,0.231178
50304784,29.580893,77.965053,0.782394,0.83209,0.910056,0.873006,0.77849,0.450681,0.475242,0.529412,0.552297,0.510086,0.554027,0.464185,0.46389,0.453392


In [5]:
import pandas as pd

# List of columns to convert to numerical types
columns_to_convert = [
    'timestep_1', 'timestep_2', 'timestep_3', 'timestep_4', 'timestep_5', 
    'timestep_6', 'timestep_7', 'timestep_8', 'timestep_9', 'timestep_10', 
    'timestep_11', 'timestep_12', 'timestep_13', 'timestep_14'
]

# Convert the specified columns to numerical types
data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Create a new DataFrame with the updated values
new_df = data.copy()

# Display the new DataFrame
print(new_df.head())


C:\Users\Shivika\AppData\Local\Temp\ipykernel_16480\3404494950.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[columns_to_convert] = data[columns_to_convert].apply(pd.to_numeric, errors='coerce')


       latitude  longitude  timestep_1  timestep_2  timestep_3  timestep_4  \
2689  30.240077  77.955621    0.524752    0.553589    0.522286    0.592040   
2690  30.240077  77.955711    0.523729    0.540253    0.470952    0.520866   
2692  30.240077  77.955890    0.467526    0.496446    0.454775    0.590361   
2693  30.240077  77.955980    0.368110    0.460727    0.458280    0.569362   
9547  30.239987  77.955890    0.574109    0.549407    0.537442    0.651181   

      timestep_5  timestep_6  timestep_7  timestep_8  timestep_9  timestep_10  \
2689    0.300691    0.313164    0.567788    0.483366    0.540415     0.497820   
2690    0.257500    0.313164    0.525828    0.478057    0.517939     0.461378   
2692    0.243655    0.300000    0.638424    0.620010    0.657407     0.640175   
2693    0.138039    0.217753    0.563296    0.518771    0.569691     0.543260   
9547    0.438735    0.406828    0.725471    0.628253    0.658680     0.610591   

      timestep_11  timestep_12  timestep_13 

In [6]:
new_df

,latitude,longitude,timestep_1,timestep_2,timestep_3,timestep_4,timestep_5,timestep_6,timestep_7,timestep_8,timestep_9,timestep_10,timestep_11,timestep_12,timestep_13,timestep_14
2689,30.240077,77.955621,0.524752,0.553589,0.522286,0.592040,0.300691,0.313164,0.567788,0.483366,0.540415,0.497820,0.447154,0.479344,0.312207,0.380660
2690,30.240077,77.955711,0.523729,0.540253,0.470952,0.520866,0.257500,0.313164,0.525828,0.478057,0.517939,0.461378,0.426923,0.532698,0.319149,0.424261
2692,30.240077,77.955890,0.467526,0.496446,0.454775,0.590361,0.243655,0.300000,0.638424,0.620010,0.657407,0.640175,0.560887,0.571265,0.380217,0.629849
2693,30.240077,77.955980,0.368110,0.460727,0.458280,0.569362,0.138039,0.217753,0.563296,0.518771,0.569691,0.543260,0.406413,0.456286,0.336175,0.436864
9547,30.239987,77.955890,0.574109,0.549407,0.537442,0.651181,0.438735,0.406828,0.725471,0.628253,0.658680,0.610591,0.486381,0.545975,0.417304,0.619677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,29.580983,77.971431,0.465719,0.540412,0.662150,0.683480,0.573653,0.366557,0.473824,0.467172,0.241561,0.195604,0.236601,0.145597,0.084599,0.150090
50298002,29.580983,77.971611,0.474039,0.578114,0.665991,0.697608,0.577744,0.381807,0.450522,0.446207,0.287449,0.236979,0.270392,0.131527,0.082659,0.173953
50298014,29.580983,77.972689,0.243697,0.242347,0.280808,0.365385,0.501665,0.422971,0.668260,0.757663,0.735261,0.712784,0.654444,0.215040,0.280859,0.231178
50304784,29.580893,77.965053,0.782394,0.832090,0.910056,0.873006,0.778490,0.450681,0.475242,0.529412,0.552297,0.510086,0.554027,0.464185,0.463890,0.453392


In [7]:
rename_dict = {f'timestep_{i}': f'Band_{i}' for i in range(1, 15)}

# Rename the columns
new_df.rename(columns=rename_dict, inplace=True)

In [8]:
Range = new_df[(new_df.Band_1 >= 0) & (new_df.Band_1 <= 1) 
            & (new_df.Band_2 >= 0) & (new_df.Band_2 <= 1) 
            & (new_df.Band_3 >= 0) & (new_df.Band_3 <= 1)
            & (new_df.Band_4 >= 0) & (new_df.Band_4 <= 1)
            & (new_df.Band_5 >= 0) & (new_df.Band_5 <= 1)
            & (new_df.Band_6 >= 0) & (new_df.Band_6 <= 1)
            & (new_df.Band_7 >= 0) & (new_df.Band_7 <= 1)
            & (new_df.Band_8 >= 0) & (new_df.Band_8 <= 1)
            & (new_df.Band_9 >= 0) & (new_df.Band_9 <= 1)
            & (new_df.Band_10 >= 0) & (new_df.Band_10 <= 1)
            & (new_df.Band_11 >= 0) & (new_df.Band_11 <= 1)
            & (new_df.Band_12 >= 0) & (new_df.Band_12 <= 1)
            & (new_df.Band_13 >= 0) & (new_df.Band_13 <= 1)
            & (new_df.Band_14 >= 0) & (new_df.Band_14 <= 1)
            
           ]

In [9]:
Range

,latitude,longitude,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_9,Band_10,Band_11,Band_12,Band_13,Band_14
2689,30.240077,77.955621,0.524752,0.553589,0.522286,0.592040,0.300691,0.313164,0.567788,0.483366,0.540415,0.497820,0.447154,0.479344,0.312207,0.380660
2690,30.240077,77.955711,0.523729,0.540253,0.470952,0.520866,0.257500,0.313164,0.525828,0.478057,0.517939,0.461378,0.426923,0.532698,0.319149,0.424261
2692,30.240077,77.955890,0.467526,0.496446,0.454775,0.590361,0.243655,0.300000,0.638424,0.620010,0.657407,0.640175,0.560887,0.571265,0.380217,0.629849
2693,30.240077,77.955980,0.368110,0.460727,0.458280,0.569362,0.138039,0.217753,0.563296,0.518771,0.569691,0.543260,0.406413,0.456286,0.336175,0.436864
9547,30.239987,77.955890,0.574109,0.549407,0.537442,0.651181,0.438735,0.406828,0.725471,0.628253,0.658680,0.610591,0.486381,0.545975,0.417304,0.619677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,29.580983,77.971431,0.465719,0.540412,0.662150,0.683480,0.573653,0.366557,0.473824,0.467172,0.241561,0.195604,0.236601,0.145597,0.084599,0.150090
50298002,29.580983,77.971611,0.474039,0.578114,0.665991,0.697608,0.577744,0.381807,0.450522,0.446207,0.287449,0.236979,0.270392,0.131527,0.082659,0.173953
50298014,29.580983,77.972689,0.243697,0.242347,0.280808,0.365385,0.501665,0.422971,0.668260,0.757663,0.735261,0.712784,0.654444,0.215040,0.280859,0.231178
50304784,29.580893,77.965053,0.782394,0.832090,0.910056,0.873006,0.778490,0.450681,0.475242,0.529412,0.552297,0.510086,0.554027,0.464185,0.463890,0.453392


In [10]:
wheat = Range.drop(['longitude','latitude'],axis=1)
wheat

,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_9,Band_10,Band_11,Band_12,Band_13,Band_14
2689,0.524752,0.553589,0.522286,0.592040,0.300691,0.313164,0.567788,0.483366,0.540415,0.497820,0.447154,0.479344,0.312207,0.380660
2690,0.523729,0.540253,0.470952,0.520866,0.257500,0.313164,0.525828,0.478057,0.517939,0.461378,0.426923,0.532698,0.319149,0.424261
2692,0.467526,0.496446,0.454775,0.590361,0.243655,0.300000,0.638424,0.620010,0.657407,0.640175,0.560887,0.571265,0.380217,0.629849
2693,0.368110,0.460727,0.458280,0.569362,0.138039,0.217753,0.563296,0.518771,0.569691,0.543260,0.406413,0.456286,0.336175,0.436864
9547,0.574109,0.549407,0.537442,0.651181,0.438735,0.406828,0.725471,0.628253,0.658680,0.610591,0.486381,0.545975,0.417304,0.619677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,0.465719,0.540412,0.662150,0.683480,0.573653,0.366557,0.473824,0.467172,0.241561,0.195604,0.236601,0.145597,0.084599,0.150090
50298002,0.474039,0.578114,0.665991,0.697608,0.577744,0.381807,0.450522,0.446207,0.287449,0.236979,0.270392,0.131527,0.082659,0.173953
50298014,0.243697,0.242347,0.280808,0.365385,0.501665,0.422971,0.668260,0.757663,0.735261,0.712784,0.654444,0.215040,0.280859,0.231178
50304784,0.782394,0.832090,0.910056,0.873006,0.778490,0.450681,0.475242,0.529412,0.552297,0.510086,0.554027,0.464185,0.463890,0.453392


In [11]:
new_column_names = [4, 19, 24, 34, 39, 49, 99, 114, 124, 129, 132, 159, 169, 184]

# Rename columns
wheat.columns = new_column_names

In [12]:
wheat

,4,19,24,34,39,49,99,114,124,129,132,159,169,184
2689,0.524752,0.553589,0.522286,0.592040,0.300691,0.313164,0.567788,0.483366,0.540415,0.497820,0.447154,0.479344,0.312207,0.380660
2690,0.523729,0.540253,0.470952,0.520866,0.257500,0.313164,0.525828,0.478057,0.517939,0.461378,0.426923,0.532698,0.319149,0.424261
2692,0.467526,0.496446,0.454775,0.590361,0.243655,0.300000,0.638424,0.620010,0.657407,0.640175,0.560887,0.571265,0.380217,0.629849
2693,0.368110,0.460727,0.458280,0.569362,0.138039,0.217753,0.563296,0.518771,0.569691,0.543260,0.406413,0.456286,0.336175,0.436864
9547,0.574109,0.549407,0.537442,0.651181,0.438735,0.406828,0.725471,0.628253,0.658680,0.610591,0.486381,0.545975,0.417304,0.619677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50298000,0.465719,0.540412,0.662150,0.683480,0.573653,0.366557,0.473824,0.467172,0.241561,0.195604,0.236601,0.145597,0.084599,0.150090
50298002,0.474039,0.578114,0.665991,0.697608,0.577744,0.381807,0.450522,0.446207,0.287449,0.236979,0.270392,0.131527,0.082659,0.173953
50298014,0.243697,0.242347,0.280808,0.365385,0.501665,0.422971,0.668260,0.757663,0.735261,0.712784,0.654444,0.215040,0.280859,0.231178
50304784,0.782394,0.832090,0.910056,0.873006,0.778490,0.450681,0.475242,0.529412,0.552297,0.510086,0.554027,0.464185,0.463890,0.453392


## jointing dates 

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Double Gaussian function
def double_gaussian(x, a1, b1, c1, a2, b2, c2):
    return (a1 * np.exp(-(x - b1)**2 / (2 * c1**2)) +
            a2 * np.exp(-(x - b2)**2 / (2 * c2**2)))

# First derivative function
def first_derivative(x, a1, b1, c1, a2, b2, c2):
    return ((-(x - b1) / c1**2) * a1 * np.exp(-(x - b1)**2 / (2 * c1**2)) +
            (-(x - b2) / c2**2) * a2 * np.exp(-(x - b2)**2 / (2 * c2**2)))


days = np.array(wheat.columns, dtype=float)  # Convert column headers to float
data = wheat.values.astype(float)  # Convert the data values to float

# Normalize the data
data_norm = (data - np.min(data, axis=1, keepdims=True)) / (np.max(data, axis=1, keepdims=True) - np.min(data, axis=1, keepdims=True))

# Fit the double Gaussian to each row
fitted_params = []
first_peak_first_deriv_days = []
for i in range(data_norm.shape[0]):
    try:
        # Adjust initial guesses based on data characteristics
        a1_guess = max(data_norm[i])
        b1_guess = days[np.argmax(data_norm[i])]
        c1_guess = (days[-1] - days[0]) / 10  # Rough estimate for the width of the first peak
        a2_guess = max(data_norm[i]) / 2
        b2_guess = days[len(days) // 2]
        c2_guess = (days[-1] - days[0]) / 5  # Rough estimate for the width of the second peak

        bounds = (0, [np.inf, np.max(days), np.inf, np.inf, np.max(days), np.inf])
        popt, _ = curve_fit(double_gaussian, days, data_norm[i], p0=[a1_guess, b1_guess, c1_guess, a2_guess, b2_guess, c2_guess], bounds=bounds, maxfev=10000)
        fitted_params.append(popt)
        
        # Calculate the first derivative
        first_deriv = first_derivative(days, *popt)
        
        # Find the first peak of the first derivative after 50 days
        threshold_day = 30
        start_index = np.where(days >= threshold_day)[0][0]
        first_peak_first_deriv_index = start_index + np.argmax(first_deriv[start_index:])
        first_peak_first_deriv_day = days[first_peak_first_deriv_index]
        first_peak_first_deriv_days.append(first_peak_first_deriv_day)
        
    except RuntimeError as e:
        # print(f"Fit did not converge for row {i+1}: {e}")
        fitted_params.append([np.nan] * 6)  # Or handle as appropriate
        first_peak_first_deriv_days.append(np.nan)
    except Exception as e:
        print(f"An error occurred for row {i+1}: {e}")
        fitted_params.append([np.nan] * 6)  # Or handle as appropriate
        first_peak_first_deriv_days.append(np.nan)

# Save first_peak_first_deriv_days to an Excel file
peak_days_df = pd.DataFrame({
    'Row': np.arange(1, len(first_peak_first_deriv_days) + 1),
    'First_Peak_First_Deriv_Day': first_peak_first_deriv_days
})


peak_days_df.to_parquet('Jointing.parquet', index=False)
print("file save into parquet")


NameError: name 'wheat' is not defined